<a href="https://colab.research.google.com/github/alexeybondarev/Deep_learning_exercises/blob/main/DL_2_HW_Bondarev_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Embeddings

Привет! В этом домашнем задании мы с помощью эмбеддингов решим задачу семантической классификации твитов.

Для этого мы воспользуемся предобученными эмбеддингами word2vec.

Для начала скачаем датасет для семантической классификации твитов:

In [ ]:
!gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
!unzip archive.zip

Downloading...
From: https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph
To: /content/archive.zip
84.9MB [00:00, 155MB/s]
Archive:  archive.zip
replace training.1600000.processed.noemoticon.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


Импортируем нужные библиотеки:

In [ ]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

In [ ]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные:

In [ ]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   emotion  1600000 non-null  int64 
 1   id       1600000 non-null  int64 
 2   date     1600000 non-null  object
 3   flag     1600000 non-null  object
 4   user     1600000 non-null  object
 5   text     1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [ ]:
examples = data["text"].sample(10)
print("\n".join(examples))

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


Как вилим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [ ]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [ ]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][1].lower())
print(" ".join(line))

@ jhicks i will think of you on sunday ! who are you by the way ? i am val .


In [ ]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) >= 3]
print(" ".join(filtered_line))

jhicks will think you sunday who are you the way val


In [ ]:
filtered_line

['jhicks',
 'will',
 'think',
 'you',
 'sunday',
 'who',
 'are',
 'you',
 'the',
 'way',
 'val']

Загрузим предобученную модель эмбеддингов. 

Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data.

Данная модель выдает эмбеддинги для **слов**. Строить по эмбеддингам слов эмбеддинги предложений мы будем ниже.

In [ ]:
word2vec = api.load("word2vec-google-news-300")

In [ ]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(300,)


### Посмотрим на статистику предложений

In [ ]:
def count_tokens(text, length):
  token_stat = {}
  tokenizer = nltk.WordPunctTokenizer()
  for line in text:
    line_text = tokenizer.tokenize(line.lower())
    filtered_line = [w for w in line_text if all(c not in string.punctuation for c in w) and len(w) >= length]
    token_stat[len(filtered_line)] = token_stat.get(len(filtered_line), 0) + 1  
  return token_stat

In [ ]:
token_statistics = count_tokens(dev_data["text"], 3)
print(token_statistics)

{16: 52781, 11: 69794, 14: 56504, 9: 81454, 8: 86835, 20: 27567, 4: 83765, 6: 92493, 7: 91442, 3: 71426, 5: 89970, 1: 10679, 13: 60194, 15: 53549, 17: 51466, 10: 75607, 18: 47771, 12: 64512, 2: 44056, 23: 2758, 21: 15872, 26: 78, 24: 913, 0: 443, 19: 40347, 22: 7388, 25: 270, 29: 5, 27: 28, 28: 14, 30: 16, 34: 2, 31: 1}


In [ ]:
total_lines = sum(token_statistics.values())
for key in sorted(token_statistics):
    print (key, '{:%}'.format(token_statistics[key] / total_lines))

0 0.034609%
1 0.834297%
2 3.441875%
3 5.580156%
4 6.544141%
5 7.028906%
6 7.226016%
7 7.143906%
8 6.783984%
9 6.363594%
10 5.906797%
11 5.452656%
12 5.040000%
13 4.702656%
14 4.414375%
15 4.183516%
16 4.123516%
17 4.020781%
18 3.732109%
19 3.152109%
20 2.153672%
21 1.240000%
22 0.577187%
23 0.215469%
24 0.071328%
25 0.021094%
26 0.006094%
27 0.002187%
28 0.001094%
29 0.000391%
30 0.001250%
31 0.000078%
34 0.000156%


Интересно, 4,5% текстов имеют 2 и меньше токенов после токенизации, что же там такое написано? И их вероятно из обучения имеет смысл выкинуть, хотя что делать с такими текстами при прогнозировании непонятно. С ругой стороны там могут быть предложения типа holy shit, что вполне имеет сентимент

In [ ]:
# а если не удалять знаки препинания? Что получится по токенам?
def count_tokens(text, length):
  token_stat = {}
  tokenizer = nltk.WordPunctTokenizer()
  for line in text:
    line_text = tokenizer.tokenize(line.lower())
    filtered_line = [w for w in line_text if len(w) >= length]
    token_stat[len(filtered_line)] = token_stat.get(len(filtered_line), 0) + 1  
  return token_stat

In [ ]:
token_statistics = count_tokens(dev_data["text"], 3)
total_lines = sum(token_statistics.values())
for key in sorted(token_statistics):
    print (key, '{:%}'.format(token_statistics[key] / total_lines))

0 0.002969%
1 0.661484%
2 3.122031%
3 5.273359%
4 6.222812%
5 6.726562%
6 6.912500%
7 6.911797%
8 6.708672%
9 6.337969%
10 5.893672%
11 5.467891%
12 5.125078%
13 4.748750%
14 4.458203%
15 4.223594%
16 4.136953%
17 4.057266%
18 3.876484%
19 3.499844%
20 2.556094%
21 1.621641%
22 0.871016%
23 0.373203%
24 0.141484%
25 0.045234%
26 0.014922%
27 0.004531%
28 0.001797%
29 0.000625%
30 0.001328%
34 0.000078%
35 0.000078%
42 0.000078%


In [ ]:
# проверим теперь кто имеет имбеддинги
def count_embeddings(text, length):
  emb_stat = {}
  tokenizer = nltk.WordPunctTokenizer()
  for line in text:
    line_text = tokenizer.tokenize(line.lower())
    filtered_line = [w for w in line_text if all(c not in string.punctuation for c in w) and (len(w) >= length) and (w in word2vec)]
    emb_stat[len(filtered_line)] = emb_stat.get(len(filtered_line), 0) + 1  
  return emb_stat

In [ ]:
embedding_statistics = count_embeddings(dev_data["text"], 3)
total_lines = sum(embedding_statistics.values())
for key in sorted(embedding_statistics):
    print (key, '{:%}'.format(embedding_statistics[key] / total_lines))

0 0.596719%
1 2.172266%
2 5.050937%
3 6.722578%
4 7.471172%
5 7.668047%
6 7.637422%
7 7.211328%
8 6.761563%
9 6.273984%
10 5.725859%
11 5.333438%
12 4.904141%
13 4.648906%
14 4.347734%
15 4.111016%
16 3.916719%
17 3.323125%
18 2.573906%
19 1.744531%
20 1.010469%
21 0.487188%
22 0.201328%
23 0.071406%
24 0.021562%
25 0.007109%
26 0.002500%
27 0.000703%
28 0.000937%
30 0.001172%
31 0.000078%
34 0.000156%


Нулевые очевидно выкидываем, с остальными попробуем работать, даже с 1 словом в предложении можно что-то сделать, если это будет слово типа fuck

Нормализуем эмбеддинги, прежде чем обучать на них сеть. 
(наверное, вы помните, что нейронные сети гораздо лучше обучаются на нормализованных данных)

Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [ ]:
from torch.utils.data import Dataset, random_split


class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])
        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        tokenizer = nltk.WordPunctTokenizer()
        line = tokenizer.tokenize(text.lower())
        filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) >= 3]
    
        return filtered_line

    def get_embeddings_(self, tokens):
        embeddings = [(word2vec.get_vector(w) - self.mean) / self.std for w in tokens if w in word2vec]

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

In [ ]:
len(dev)

1280000

Отлично, мы готовы с помощью эмбеддингов слов превращать твиты в векторы и обучать нейронную сеть.

Превращать твиты в векторы, используя эмбеддинги слов, можно несколькими способами. А именно такими:

## Average embedding (2 балла)
---
Это самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предлоежнии (которые остались после токенизации и удаления коротких слов, конечно). 

In [ ]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


Давайте сделаем визуализацию полученных векторов твитов тренировочного (dev) датасета. Так мы увидим, насколько хорошо твиты с разными target значениями отделяются друг от друга, т.е. насколько хорошо усреднение эмбеддингов слов предложения передает информацию о предложении.

Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Если хотите, можете вместо PCA использовать TSNE: так у вас получится более точная проекция на плоскость (а значит, более информативная, т.е. отражающая реальное положение векторов твитов в пространстве). Но TSNE будет работать намного дольше.

In [ ]:
from sklearn.decomposition import PCA


pca = PCA(n_components=2)
examples["transformed_features"] = pca.fit_transform(examples["features"])

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [ ]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1108', ...)

Скорее всего, на визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [ ]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.sum(b["feature"], axis=0) for b in batch] # кажется, здесь надо mean на sum заменить
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

Определим функции для тренировки и теста модели:

In [ ]:
from tqdm.notebook import tqdm


def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)

        # Получи предсказания модели
        y_pred = model(features)
        loss = criterion(y_pred, targets)
        # Обнови параметры модели
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
    

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)

            # Получи предсказания модели
            y_pred = model(features)
            loss = criterion(y_pred, targets)
            score, predicted = torch.max(y_pred, 1)
            acc = (targets == predicted).sum().float() / len(targets)

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию. Вы можете сами выбрать количество слоев в нейронной сети, ваш любимый оптимизатор и целевую функцию.


In [ ]:
import torch.nn as nn
from torch.optim import Adam


# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-3
num_epochs = 1

# N - размер батча (batch_size, нужно для метода оптимизации); 
# D_in - размерность входа (количество признаков у объекта);
# H - размерность скрытых слоёв; 
# D_out - размерность выходного слоя (суть - количество классов)
D_in, H1, H2, H3, D_out = vector_size, 50, 50, 10, num_classes

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H1),
    torch.nn.Tanh(),
    torch.nn.Linear(H1, H2),
    torch.nn.ReLU(),
    torch.nn.Linear(H2, H3),
    torch.nn.Tanh(),
    torch.nn.Linear(H3, D_out),
)
model.to(device)
criterion = torch.nn.CrossEntropyLoss(size_average=False)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Наконец, обучим модель и протестируем её.

После каждой эпохи будем проверять качество модели на валидационной части датасета. Если метрика стала лучше, будем сохранять модель. **Подумайте, какая метрика (точность или лосс) будет лучше работать в этой задаче?** 

In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 501.93104431152346, 'Test Acc': 0.7611875}


In [ ]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 503.50060303904377, 'Test Acc': 0.7592539436900958}


Трехслойная полносвязная нейросеть дает порядка 75%, что в общем не очень хорошо.

## Embeddings for unknown words (8 баллов)

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

- Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. (4 балла)
- Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf. (4 балла)

Реализуйте оба варианта **ниже**. Напишите, какой способ сработал лучше и ваши мысли, почему так получилось.

In [ ]:
class TwitterDatasetAdvanced(TwitterDataset):
  def get_embeddings_(self, tokens):
        embeddings = []
        for i, w in enumerate(tokens):
          if w in word2vec:
            embeddings.append((word2vec.get_vector(w) - self.mean) / self.std)
          else:
            embedding_neighbours = np.zeros(300, dtype=float)
            if len(tokens) == 2:
              if i == 0: 
                neighbour_word = tokens[i+1]
                if neighbour_word in word2vec:
                  embedding_neighbours += word2vec.get_vector(neighbour_word)
              else:
                neighbour_word = tokens[i-1]
                if neighbour_word in word2vec:
                  embedding_neighbours += word2vec.get_vector(neighbour_word)
            if (i < 1) and (len(tokens) >= 3):
              neighbour_word = tokens[i+1]
              if neighbour_word in word2vec:
               embedding_neighbours += word2vec.get_vector(neighbour_word)
            
            if (i >= len(tokens) - 1) and (len(tokens) >= 3):
              neighbour_word = tokens[i-1]
              if neighbour_word in word2vec:
                embedding_neighbours += word2vec.get_vector(neighbour_word)
            if 1 <= i < len(tokens) - 1: 
              neighbour_word = tokens[i-1]
              if neighbour_word in word2vec:
                embedding_neighbours += word2vec.get_vector(neighbour_word)
              neighbour_word = tokens[i+1]
              if neighbour_word in word2vec:
                embedding_neighbours += word2vec.get_vector(neighbour_word)
            if sum(embedding_neighbours) != 0:
              embeddings.append((embedding_neighbours - self.mean) / self.std) 
        
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

In [ ]:
dev = TwitterDatasetAdvanced(dev_data, "text", "emotion", word2vec)

In [ ]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.sum(b["feature"], axis=0) for b in batch] # кажется, здесь надо mean на sum заменить
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [ ]:
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-3
num_epochs = 1

# N - размер батча (batch_size, нужно для метода оптимизации); 
# D_in - размерность входа (количество признаков у объекта);
# H - размерность скрытых слоёв; 
# D_out - размерность выходного слоя (суть - количество классов)
D_in, H1, H2, H3, D_out = vector_size, 50, 50, 10, num_classes

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H1),
    torch.nn.Tanh(),
    torch.nn.Linear(H1, H2),
    torch.nn.ReLU(),
    torch.nn.Linear(H2, H3),
    torch.nn.Tanh(),
    torch.nn.Linear(H3, D_out),
)

model.to(device)

criterion = torch.nn.CrossEntropyLoss(size_average=False)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 509.6139049072266, 'Test Acc': 0.756265625}


In [ ]:
test_loader = DataLoader(
    TwitterDatasetAdvanced(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 508.42380735744683, 'Test Acc': 0.7552041733226837}


## В пределах погрешности точность на тестовых данных не изменилась при добавлении векторов отсутствующих в словаре слов, результат в районе 76%

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer()
dev_data_tfidf = vectorizer.fit_transform(dev_data["text"])


In [ ]:
print(dev_data_tfidf.shape)

(1280000, 589604)


In [ ]:
names = vectorizer.get_feature_names()

### <font color="red"> Все, что ниже пока не заработало :( Слишком долго считается, нужен какой-то другой подход 

In [ ]:
# модифицируем подсчет эмбеддингов, чтобы учитывать слова с весами из Tf-idf
class TwitterDatasetTFIDF(TwitterDataset):
  def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])
        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens, item)
        
        return {"feature": embeddings, "target": label}
 
  def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec, tfidf, names):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

        self.tfidf = tfidf
        self.names = names
  
  def get_embeddings_(self, tokens, item):
        embeddings = [(word2vec.get_vector(w) - self.mean) / self.std * self.tfidf[item, self.names.index(w)] 
                      for w in tokens if w in word2vec and w in self.names]

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

In [ ]:
dev = TwitterDatasetTFIDF(dev_data, "text", "emotion", word2vec, dev_data_tfidf, names)

In [ ]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.sum(b["feature"], axis=0) for b in batch] # кажется, здесь надо mean на sum заменить
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [ ]:
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-3
num_epochs = 1

# N - размер батча (batch_size, нужно для метода оптимизации); 
# D_in - размерность входа (количество признаков у объекта);
# H - размерность скрытых слоёв; 
# D_out - размерность выходного слоя (суть - количество классов)
D_in, H1, H2, H3, D_out = vector_size, 50, 50, 10, num_classes

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H1),
    torch.nn.Tanh(),
    torch.nn.Linear(H1, H2),
    torch.nn.ReLU(),
    torch.nn.Linear(H2, H3),
    torch.nn.Tanh(),
    torch.nn.Linear(H3, D_out),
)

model.to(device)

criterion = torch.nn.CrossEntropyLoss(size_average=False)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

KeyboardInterrupt: ignored